# Twitter API를 통해 Crypto tweet 수집

- API 튜토리얼 해보기
- BTC, ETH만 수집

특이사항: 
- 일론머스크 인수 후 (X로 개명) free tier에서 retrieve 불가. 
    - 할 수 없이 `twikit` 이라는 scraping 기반 라이브러리 사용
    - 주의 사항: 익명 temp 계정으로 만든 것만 사용할 것. 

In [1]:
import os, sys, warnings
from pathlib import Path

In [2]:
# warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import asyncio
import nest_asyncio
from twikit import Client

In [5]:
nest_asyncio.apply()

In [6]:
%load_ext dotenv
%dotenv

## `twikit` tutorial 

Twitter API 쓰지 않고 scraping 방식으로 tweet 불러옴. 

구글에서 advanced search 로 날짜 넣어 검색하듯, 트위터 검색창에 날짜 넣어 검색하는 식으로 트윗을 모아볼 수 있음. 

한 번 로드할 때 최대 20개만 되기 때문에 최대한 잘게 쪼개 1일씩 가져와 데이터를 생성해야 함. 

일단 테스트 용으로 5월1일 ~ 7월 10일까지의 데이터만 생성해 보겠음. 

### 기본 사용법

In [7]:
client = Client('en-US')

In [8]:
async def main():
    await client.login(
        auth_info_1=os.getenv('TWITTER_USERNAME'),
        auth_info_2=os.getenv('TWITTER_EMAIL'),
        password=os.getenv('TWITTER_PASSWORD'),
    )

In [9]:
asyncio.run(main())

In [30]:
tweets = await client.search_tweet('$ETH since:2023-05-05 until:2023-05-06', 'Latest', count=20)

In [31]:
tweets

[<Tweet id="1654636966367961088">, <Tweet id="1654636966338494467">, <Tweet id="1654636933266432003">, <Tweet id="1654636867051028480">, <Tweet id="1654636838307364864">, <Tweet id="1654636741616099328">, <Tweet id="1654636719998910465">, <Tweet id="1654636693033570305">, <Tweet id="1654636672112271360">, <Tweet id="1654636597113942016">]

In [32]:
len(tweets)

10

In [33]:
t = tweets[0]

In [34]:
t.created_at

'Fri May 05 23:59:29 +0000 2023'

In [35]:
t.text

'🟩 $ETH block trade alert [Deribit] 🧱🧱 \n\nMAY(26) $2,000 Calls bought 250 @ 0.0466 ETH($93.01); Total=11.6500 ETH($23,251.30); IV=50.13%; OI=44331; Index=$1,995.82'

### 날짜 범위를 주면 그만큼 데이터를 가져오는 함수 추가. 

혹시 모를 ban을 위해 한 번 가져올 때 3초씩 sleep을 주겠음. 

In [ ]:
def get_tweets_from_date(date):
    # date format should be 'YYYY-MM-DD'

    since = pd.to_datetime(date) - 1

In [36]:
pd.to_datetime('2013-03-02') - 1

TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`